In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

usage: conda-script.py [-h] [-V] command ...
conda-script.py: error: unrecognized arguments: # uncomment this line if you haven't completed the Foursquare API lab
usage: conda-script.py [-h] [-V] command ...
conda-script.py: error: unrecognized arguments: # uncomment this line if you haven't completed the Foursquare API lab


Libraries imported.


In [101]:
df=pd.read_csv('Mumbai_Pincode.csv')

In [102]:
df.head()

,Pincode,Place,State,City,Latitude,Longitude
0,400001,Mumbai G.P.O.,Maharashtra,Mumbai,18.629,72.892
1,400001,Bazargate,Maharashtra,Mumbai,18.629,72.892
2,400001,Town Hall (Mumbai),Maharashtra,Mumbai,18.629,72.892
3,400001,Tajmahal,Maharashtra,Mumbai,18.629,72.892
4,400001,Stock Exchange,Maharashtra,Mumbai,18.629,72.892


In [103]:
address = 'Mumbai'

geolocator = Nominatim(user_agent="Mumbai_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Mumbai are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Mumbai are 18.9387711, 72.8353355.


In [104]:
map_Mumbai = folium.Map(location=[latitude, longitude], zoom_start=12)
map_Mumbai

In [105]:
CLIENT_ID = '1GTOB151OXYG0KP0THG4BS5I1GEB0DLNMG5WZ04NJBXBKXDX' # your Foursquare ID
CLIENT_SECRET = 'KH2DA45DVKE1SAKITIGO0LW3URRMDJTXNJXH4FAXH53GRHQD' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 1GTOB151OXYG0KP0THG4BS5I1GEB0DLNMG5WZ04NJBXBKXDX
CLIENT_SECRET:KH2DA45DVKE1SAKITIGO0LW3URRMDJTXNJXH4FAXH53GRHQD


In [106]:
df.loc[0,'Place']

'Mumbai G.P.O.'

In [107]:
df.head()

,Pincode,Place,State,City,Latitude,Longitude
0,400001,Mumbai G.P.O.,Maharashtra,Mumbai,18.629,72.892
1,400001,Bazargate,Maharashtra,Mumbai,18.629,72.892
2,400001,Town Hall (Mumbai),Maharashtra,Mumbai,18.629,72.892
3,400001,Tajmahal,Maharashtra,Mumbai,18.629,72.892
4,400001,Stock Exchange,Maharashtra,Mumbai,18.629,72.892


In [108]:
df.loc[0,'City']

'Mumbai'

In [109]:
df.columns = df.columns.str.strip()

In [110]:
# add markers to map
for lat, lng, label in zip(df['Latitude'], df['Longitude'], df['Place']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Mumbai)  
    
map_Mumbai

In [111]:
Place_latitude =df.loc[0, 'Latitude']
Place_longitude = df.loc[0, 'Longitude'] 

Place_name = df.loc[0, 'Place'] 

print('Latitude and longitude values of {} are {}, {}.'.format(Place_name, 
                                                               Place_latitude, 
                                                               Place_longitude))

Latitude and longitude values of Mumbai G.P.O. are 18.629, 72.892.


Now, let's get the top 100 venues that are in Mumbai G.P.O within a radius of 500 meters.
First, let's create the GET request URL. Name your URL url.

In [112]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    Place_latitude, 
    Place_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=1GTOB151OXYG0KP0THG4BS5I1GEB0DLNMG5WZ04NJBXBKXDX&client_secret=KH2DA45DVKE1SAKITIGO0LW3URRMDJTXNJXH4FAXH53GRHQD&v=20180605&ll=18.629,72.892&radius=500&limit=100'

In [113]:
results = requests.get(url).json()

In [114]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Structuring the json into a pandas dataframe

In [115]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Shirke's Holiday Home,Resort,18.629468,72.890207
1,ACHI BIZ SERVICES PTE. LTD.,Business Service,18.630000,72.891530
2,Kadam Holidays,Hotel,18.629150,72.888194


Repeat the same process to all the Places in Mumbai

In [116]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Run the above function on each place and create a new dataframe called Mumbai_Venues.

In [117]:
Mumbai_venues = getNearbyVenues(names=df['Place'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

Mumbai G.P.O.
Bazargate
Town Hall (Mumbai)
Tajmahal
Stock Exchange
M.P.T.
Kalbadevi
S. C. Court
Ramwadi
Thakurdwar
Mandvi (Mumbai)
Null Bazar
B.P.Lane
Masjid
Girgaon
Opera House
Ambewadi (Mumbai)
Charni Road
Chaupati
Madhavbaug
Colaba
Holiday Camp
V.W.T.C.
Colaba Bazar
Asvini
Malabar Hill
Tardeo
Grant Road
S V Marg
N.S.Patkar Marg
Bharat Nagar (Mumbai)
Falkland Road
Mumbai Central
M A Marg
J.J.Hospital
Kamathipura
Noor Baug
Princess Dock
Chinchbunder
Mazgaon
Mazgaon Road
V K Bhavan
Mazgaon Dock
Dockyard Road
Agripada
Chinchpokli
Jacob Circle
Haines Road
Parel
Chamarbaug
Haffkin Institute
Lal Baug
Parel Naka
Parel Rly Work Shop
BEST STaff Quarters
Delisle Road
Naigaon (Mumbai)
Dadar
Dadar Colony
Sewri
Mahim
Mahim Bazar
Mori Road
Kapad Bazar
Mahim East
Dharavi Road
Dharavi
Worli
Worli Naka
Matunga
Marine Lines
Churchgate
Central Building
Nariman Point
New Yogakshema
Sion
Raoli Camp
Transit Camp
Chunabhatti
Nehru Nagar (Mumbai)
Prabhadevi
New Prabhadevi Road
Cumballa Sea Face
Gowalia Tank

Checking the resulting dataframe

In [118]:
Mumbai_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Mumbai G.P.O.,18.629,72.892,Shirke's Holiday Home,18.629468,72.890207,Resort
1,Mumbai G.P.O.,18.629,72.892,ACHI BIZ SERVICES PTE. LTD.,18.630000,72.891530,Business Service
2,Mumbai G.P.O.,18.629,72.892,Kadam Holidays,18.629150,72.888194,Hotel
3,Bazargate,18.629,72.892,Shirke's Holiday Home,18.629468,72.890207,Resort
4,Bazargate,18.629,72.892,ACHI BIZ SERVICES PTE. LTD.,18.630000,72.891530,Business Service


Venues returned for each neighborhood

In [119]:
Mumbai_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
A I Staff Colony,8,8,8,8,8,8
Aareymilk Colony,4,4,4,4,4,4
Agripada,3,3,3,3,3,3
Airport (Mumbai),7,7,7,7,7,7
Ambewadi (Mumbai),18,18,18,18,18,18
Andheri,4,4,4,4,4,4
Andheri East,9,9,9,9,9,9
Andheri Railway Station,2,2,2,2,2,2
Antop Hill,13,13,13,13,13,13


In [120]:
print('There are {} uniques categories.'.format(len(Mumbai_venues['Venue Category'].unique())))

There are 116 uniques categories.


Analysing each neighborhood

In [121]:
# one hot encoding
Mumbai_onehot = pd.get_dummies(Mumbai_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Mumbai_onehot['Neighborhood'] = Mumbai_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Mumbai_onehot.columns[-1]] + list(Mumbai_onehot.columns[:-1])
Mumbai_onehot = Mumbai_onehot[fixed_columns]

Mumbai_onehot.head()

,Neighborhood,Afghan Restaurant,American Restaurant,Aquarium,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,Beach,Bengali Restaurant,Bike Rental / Bike Share,Bistro,Bookstore,Breakfast Spot,Brewery,Burger Joint,Bus Station,Business Service,Café,Chaat Place,Cheese Shop,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Convenience Store,Cosmetics Shop,Cupcake Shop,Dance Studio,Department Store,Dessert Shop,Diner,Dive Bar,Donut Shop,Electronics Store,Event Space,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Garden,Gastropub,Gift Shop,Gourmet Shop,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Harbor / Marina,History Museum,Home Service,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indoor Play Area,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Lake,Light Rail Station,Lounge,Market,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Movie Theater,Multiplex,Nightclub,Other Great Outdoors,Performing Arts Venue,Pizza Place,Playground,Plaza,Pub,Racetrack,Recreation Center,Resort,Rest Area,Restaurant,Salad Place,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shopping Mall,Smoke Shop,Snack Place,Spa,Spanish Restaurant,Steakhouse,Supermarket,Surf Spot,Tea Room,Tennis Court,Thai Restaurant,Theater,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Wine Bar,Women's Store,Yoga Studio
0,Mumbai G.P.O.,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Mumbai G.P.O.,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Mumbai G.P.O.,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Bazargate,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Bazargate,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [122]:
Mumbai_onehot.shape

(2087, 117)

Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [123]:
Mumbai_grouped = Mumbai_onehot.groupby('Neighborhood').mean().reset_index()
Mumbai_grouped

,Neighborhood,Afghan Restaurant,American Restaurant,Aquarium,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,Beach,Bengali Restaurant,Bike Rental / Bike Share,Bistro,Bookstore,Breakfast Spot,Brewery,Burger Joint,Bus Station,Business Service,Café,Chaat Place,Cheese Shop,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Convenience Store,Cosmetics Shop,Cupcake Shop,Dance Studio,Department Store,Dessert Shop,Diner,Dive Bar,Donut Shop,Electronics Store,Event Space,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Garden,Gastropub,Gift Shop,Gourmet Shop,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Harbor / Marina,History Museum,Home Service,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indoor Play Area,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Lake,Light Rail Station,Lounge,Market,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Movie Theater,Multiplex,Nightclub,Other Great Outdoors,Performing Arts Venue,Pizza Place,Playground,Plaza,Pub,Racetrack,Recreation Center,Resort,Rest Area,Restaurant,Salad Place,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shopping Mall,Smoke Shop,Snack Place,Spa,Spanish Restaurant,Steakhouse,Supermarket,Surf Spot,Tea Room,Tennis Court,Thai Restaurant,Theater,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Wine Bar,Women's Store,Yoga Studio
0,A I Staff Colony,0.00000,0.000000,0.000000,0.125000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.125000,0.000000,0.000000,0.000000,0.000000,0.000000,0.125000,0.00,0.00,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.125000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.125,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00000,0.125,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.12500,0.00000,0.000000,0.000000,0.0,0.125,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Aareymilk Colony,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.00000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.00,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.250000,0.250000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.250000,0.000000,0.00000,0.00000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Agripada,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.00000,0.000000,0.000000,0.000000,0.333333,0.000000,0.000000,0.00,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.33

In [124]:
Mumbai_grouped.shape

(217, 117)

Printing each neighborhood along with the top 5 most common venues

In [125]:
num_top_venues = 5

for hood in Mumbai_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Mumbai_grouped[Mumbai_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----A I Staff Colony----
                        venue  freq
0           Indian Restaurant  0.12
1                    Tea Room  0.12
2                 Coffee Shop  0.12
3  Modern European Restaurant  0.12
4                        Café  0.12


----Aareymilk Colony----
                       venue  freq
0             Ice Cream Shop  0.25
1          Indian Restaurant  0.25
2       Fast Food Restaurant  0.25
3                 Smoke Shop  0.25
4  Middle Eastern Restaurant  0.00


----Agripada----
                  venue  freq
0        History Museum  0.33
1             Multiplex  0.33
2  Fast Food Restaurant  0.33
3     Afghan Restaurant  0.00
4    Mexican Restaurant  0.00


----Airport (Mumbai)----
                             venue  freq
0                       Restaurant  0.14
1                     Gourmet Shop  0.14
2                     Dessert Shop  0.14
3               Italian Restaurant  0.14
4  Molecular Gastronomy Restaurant  0.14


----Ambewadi (Mumbai)----
             venue  fr

Putting this into a dataframe

In [126]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [127]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Mumbai_grouped['Neighborhood']

for ind in np.arange(Mumbai_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Mumbai_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,A I Staff Colony,Tea Room,Café,Asian Restaurant,Spa,Coffee Shop,Indian Restaurant,Racetrack,Modern European Restaurant,Electronics Store,Farmers Market
1,Aareymilk Colony,Smoke Shop,Ice Cream Shop,Indian Restaurant,Fast Food Restaurant,Yoga Studio,Department Store,Dessert Shop,Diner,Dive Bar,Donut Shop
2,Agripada,Multiplex,Fast Food Restaurant,History Museum,Yoga Studio,Flower Shop,Department Store,Dessert Shop,Diner,Dive Bar,Donut Shop
3,Airport (Mumbai),Gym,Dessert Shop,Gourmet Shop,Italian Restaurant,Molecular Gastronomy Restaurant,Restaurant,Café,Yoga Studio,Fish & Chips Shop,Diner
4,Ambewadi (Mumbai),Harbor / Marina,Café,Train Station,Ice Cream Shop,Fast Food Restaurant,Juice Bar,Food Truck,Indian Restaurant,Gastropub,Gym


Cluster Neighborhoods using KMeans

In [128]:
# set number of clusters
kclusters = 8

Mumbai_grouped_clustering = Mumbai_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Mumbai_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 2, 2, 1, 1, 0, 0, 6, 0, 1])

In [129]:
df.head()

,Pincode,Place,State,City,Latitude,Longitude
0,400001,Mumbai G.P.O.,Maharashtra,Mumbai,18.629,72.892
1,400001,Bazargate,Maharashtra,Mumbai,18.629,72.892
2,400001,Town Hall (Mumbai),Maharashtra,Mumbai,18.629,72.892
3,400001,Tajmahal,Maharashtra,Mumbai,18.629,72.892
4,400001,Stock Exchange,Maharashtra,Mumbai,18.629,72.892


In [130]:
Mumbai_merged=df

In [131]:
Mumbai_merged.rename(columns={'Place':'Neighborhood'},inplace=True)
Mumbai_merged.head()

,Pincode,Neighborhood,State,City,Latitude,Longitude
0,400001,Mumbai G.P.O.,Maharashtra,Mumbai,18.629,72.892
1,400001,Bazargate,Maharashtra,Mumbai,18.629,72.892
2,400001,Town Hall (Mumbai),Maharashtra,Mumbai,18.629,72.892
3,400001,Tajmahal,Maharashtra,Mumbai,18.629,72.892
4,400001,Stock Exchange,Maharashtra,Mumbai,18.629,72.892


In [132]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Mumbai_merged = df
Mumbai_merged = Mumbai_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
Mumbai_merged=Mumbai_merged.dropna()
Mumbai_merged=Mumbai_merged.astype({'Cluster Labels':int})
Mumbai_merged.head() 

,Pincode,Neighborhood,State,City,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,400001,Mumbai G.P.O.,Maharashtra,Mumbai,18.629,72.892,4,Hotel,Resort,Business Service,Yoga Studio,Flower Shop,Dessert Shop,Diner,Dive Bar,Donut Shop,Electronics Store
1,400001,Bazargate,Maharashtra,Mumbai,18.629,72.892,4,Hotel,Resort,Business Service,Yoga Studio,Flower Shop,Dessert Shop,Diner,Dive Bar,Donut Shop,Electronics Store
2,400001,Town Hall (Mumbai),Maharashtra,Mumbai,18.629,72.892,4,Hotel,Resort,Business Service,Yoga Studio,Flower Shop,Dessert Shop,Diner,Dive Bar,Donut Shop,Electronics Store
3,400001,Tajmahal,Maharashtra,Mumbai,18.629,72.892,4,Hotel,Resort,Business Service,Yoga Studio,Flower Shop,Dessert Shop,Diner,Dive Bar,Donut Shop,Electronics Store
4,400001,Stock Exchange,Maharashtra,Mumbai,18.629,72.892,4,Hotel,Resort,Business Service,Yoga Studio,Flower Shop,Dessert Shop,Diner,Dive Bar,Donut Shop,Electronics Store


Visualising clusters

In [150]:
Mumbai_merged.loc[Mumbai_merged['Cluster Labels'] == 0, Mumbai_merged.columns[[1] + list(range(5, Mumbai_merged.shape[1]))]].reset_index()

,index,Neighborhood,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,26,Tardeo,72.817,0,Department Store,Indian Restaurant,Vegetarian / Vegan Restaurant,Snack Place,Restaurant,Event Space,Breakfast Spot,Juice Bar,Food Truck,Coffee Shop
1,27,Grant Road,72.817,0,Department Store,Indian Restaurant,Vegetarian / Vegan Restaurant,Snack Place,Restaurant,Event Space,Breakfast Spot,Juice Bar,Food Truck,Coffee Shop
2,28,S V Marg,72.817,0,Department Store,Indian Restaurant,Vegetarian / Vegan Restaurant,Snack Place,Restaurant,Event Space,Breakfast Spot,Juice Bar,Food Truck,Coffee Shop
3,29,N.S.Patkar Marg,72.817,0,Department Store,Indian Restaurant,Vegetarian / Vegan Restaurant,Snack Place,Restaurant,Event Space,Breakfast Spot,Juice Bar,Food Truck,Coffee Shop
4,30,Bharat Nagar (Mumbai),72.817,0,Department Store,Indian Restaurant,Vegetarian / Vegan Restaurant,Snack Place,Restaurant,Event Space,Breakfast Spot,Juice Bar,Food Truck,Coffee Shop
5,48,Parel,72.833,0,Indian Restaurant,Dessert Shop,Mediterranean Restaurant,Café,Nightclub,Cocktail Bar,Bus Station,Seafood Restaurant,Brewery,Fast Food Restaurant
6,49,Chamarbaug,72.833,0,Indian Restaurant,Dessert Shop,Mediterranean Restaurant,Café,Nightclub,Cocktail Bar,Bus Station,Seafood Restaurant,Brewery,Fast Food Restaurant
7,50,Haffkin Institute,72.833,0,Indian Restaurant,Dessert Shop,Mediterranean Restaurant,Café,Nightclub,Cocktail Bar,Bus Station,Seafood Restaurant,Brewery,Fast Food Restaurant
8,51,Lal Baug,72.833,0,Indian Restaurant,Dessert Shop,Mediterranean Restaurant,Café,Nightclub,Cocktail Bar,Bus Station,Seafood Restaurant,Brewery,Fast Food Restaurant
9,52,Parel Naka,72.833,0,Indian Restaurant,Dessert Shop,Mediterranean Restaurant,Café,Nightclub,Cocktail Bar,Bus Station,Seafood Restaurant,Brewery,Fast Food Restaurant


In [140]:
Mumbai_merged.loc[Mumbai_merged['Cluster Labels'] == 1, Mumbai_merged.columns[[1] + list(range(5, Mumbai_merged.shape[1]))]]

,Neighborhood,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Girgaon,72.817,1,Harbor / Marina,Café,Train Station,Ice Cream Shop,Fast Food Restaurant,Juice Bar,Food Truck,Indian Restaurant,Gastropub,Gym
15,Opera House,72.817,1,Harbor / Marina,Café,Train Station,Ice Cream Shop,Fast Food Restaurant,Juice Bar,Food Truck,Indian Restaurant,Gastropub,Gym
16,Ambewadi (Mumbai),72.817,1,Harbor / Marina,Café,Train Station,Ice Cream Shop,Fast Food Restaurant,Juice Bar,Food Truck,Indian Restaurant,Gastropub,Gym
17,Charni Road,72.817,1,Harbor / Marina,Café,Train Station,Ice Cream Shop,Fast Food Restaurant,Juice Bar,Food Truck,Indian Restaurant,Gastropub,Gym
18,Chaupati,72.817,1,Harbor / Marina,Café,Train Station,Ice Cream Shop,Fast Food Restaurant,Juice Bar,Food Truck,Indian Restaurant,Gastropub,Gym
19,Madhavbaug,72.817,1,Harbor / Marina,Café,Train Station,Ice Cream Shop,Fast Food Restaurant,Juice Bar,Food Truck,Indian Restaurant,Gastropub,Gym
59,Sewri,72.850,1,Cosmetics Shop,Train Station,Bus Station,Flea Market,Flower Shop,Department Store,Dessert Shop,Diner,Dive Bar,Donut Shop
60,Mahim,72.850,1,Department Store,Dessert Shop,Women's Store,Shopping Mall,Train Station,Ice Cream Shop,Tennis Court,Flea Market,Clothing Store,Chinese Restaurant
61,Mahim Bazar,72.850,1,Department Store,Dessert Shop,Women's Store,Shopping Mall,Train Station,Ice Cream Shop,Tennis Court,Flea Market,Clothing Store,Chinese Restaurant
62,Mori Road,72.850,1,Department Store,Dessert Shop,Women's Store,Shopping Mall,Train Station,Ice Cream Shop,Tennis Court,Flea Market,Clothing Store,Chinese Restaurant


In [141]:
Mumbai_merged.loc[Mumbai_merged['Cluster Labels'] == 2, Mumbai_merged.columns[[1] + list(range(5, Mumbai_merged.shape[1]))]]

,Neighborhood,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,Mandvi (Mumbai),72.833,2,Indian Restaurant,Market,Rest Area,American Restaurant,Fast Food Restaurant,Food,Cheese Shop,Electronics Store,Ice Cream Shop,Middle Eastern Restaurant
11,Null Bazar,72.833,2,Indian Restaurant,Market,Rest Area,American Restaurant,Fast Food Restaurant,Food,Cheese Shop,Electronics Store,Ice Cream Shop,Middle Eastern Restaurant
12,B.P.Lane,72.833,2,Indian Restaurant,Market,Rest Area,American Restaurant,Fast Food Restaurant,Food,Cheese Shop,Electronics Store,Ice Cream Shop,Middle Eastern Restaurant
13,Masjid,72.833,2,Indian Restaurant,Market,Rest Area,American Restaurant,Fast Food Restaurant,Food,Cheese Shop,Electronics Store,Ice Cream Shop,Middle Eastern Restaurant
44,Agripada,72.833,2,Multiplex,Fast Food Restaurant,History Museum,Yoga Studio,Flower Shop,Department Store,Dessert Shop,Diner,Dive Bar,Donut Shop
45,Chinchpokli,72.833,2,Multiplex,Fast Food Restaurant,History Museum,Yoga Studio,Flower Shop,Department Store,Dessert Shop,Diner,Dive Bar,Donut Shop
46,Jacob Circle,72.833,2,Multiplex,Fast Food Restaurant,History Museum,Yoga Studio,Flower Shop,Department Store,Dessert Shop,Diner,Dive Bar,Donut Shop
47,Haines Road,72.833,2,Multiplex,Fast Food Restaurant,History Museum,Yoga Studio,Flower Shop,Department Store,Dessert Shop,Diner,Dive Bar,Donut Shop
147,Jogeshwari East,72.850,2,Smoke Shop,Ice Cream Shop,Indian Restaurant,Fast Food Restaurant,Yoga Studio,Department Store,Dessert Shop,Diner,Dive Bar,Donut Shop
156,S R P F Camp,72.850,2,Smoke Shop,Ice Cream Shop,Indian Restaurant,Fast Food Restaurant,Yoga Studio,Department Store,Dessert Shop,Diner,Dive Bar,Donut Shop


In [142]:
Mumbai_merged.loc[Mumbai_merged['Cluster Labels'] == 3, Mumbai_merged.columns[[1] + list(range(5, Mumbai_merged.shape[1]))]]

,Neighborhood,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
201,Mulund Colony,72.949,3,Clothing Store,Movie Theater,Yoga Studio,Flower Shop,Dessert Shop,Diner,Dive Bar,Donut Shop,Electronics Store,Event Space
202,Bhandup Complex,72.949,3,Clothing Store,Movie Theater,Yoga Studio,Flower Shop,Dessert Shop,Diner,Dive Bar,Donut Shop,Electronics Store,Event Space
203,Kannamwar Nagar,72.949,3,Clothing Store,Movie Theater,Yoga Studio,Flower Shop,Dessert Shop,Diner,Dive Bar,Donut Shop,Electronics Store,Event Space
204,Tagore Nagar,72.949,3,Clothing Store,Movie Theater,Yoga Studio,Flower Shop,Dessert Shop,Diner,Dive Bar,Donut Shop,Electronics Store,Event Space
205,Barve Nagar,72.949,3,Clothing Store,Movie Theater,Yoga Studio,Flower Shop,Dessert Shop,Diner,Dive Bar,Donut Shop,Electronics Store,Event Space
206,BARC,72.949,3,Clothing Store,Movie Theater,Yoga Studio,Flower Shop,Dessert Shop,Diner,Dive Bar,Donut Shop,Electronics Store,Event Space
207,Ghatkopar West,72.949,3,Clothing Store,Movie Theater,Yoga Studio,Flower Shop,Dessert Shop,Diner,Dive Bar,Donut Shop,Electronics Store,Event Space
208,Rifle Range,72.949,3,Clothing Store,Movie Theater,Yoga Studio,Flower Shop,Dessert Shop,Diner,Dive Bar,Donut Shop,Electronics Store,Event Space
209,Sahakar Bhavan,72.949,3,Clothing Store,Movie Theater,Yoga Studio,Flower Shop,Dessert Shop,Diner,Dive Bar,Donut Shop,Electronics Store,Event Space
210,Sandeepany Sadhanalya,72.949,3,Clothing Store,Movie Theater,Yoga Studio,Flower Shop,Dessert Shop,Diner,Dive Bar,Donut Shop,Electronics Store,Event Space


In [143]:
Mumbai_merged.loc[Mumbai_merged['Cluster Labels'] == 4, Mumbai_merged.columns[[1] + list(range(5, Mumbai_merged.shape[1]))]]

,Neighborhood,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Mumbai G.P.O.,72.892,4,Hotel,Resort,Business Service,Yoga Studio,Flower Shop,Dessert Shop,Diner,Dive Bar,Donut Shop,Electronics Store
1,Bazargate,72.892,4,Hotel,Resort,Business Service,Yoga Studio,Flower Shop,Dessert Shop,Diner,Dive Bar,Donut Shop,Electronics Store
2,Town Hall (Mumbai),72.892,4,Hotel,Resort,Business Service,Yoga Studio,Flower Shop,Dessert Shop,Diner,Dive Bar,Donut Shop,Electronics Store
3,Tajmahal,72.892,4,Hotel,Resort,Business Service,Yoga Studio,Flower Shop,Dessert Shop,Diner,Dive Bar,Donut Shop,Electronics Store
4,Stock Exchange,72.892,4,Hotel,Resort,Business Service,Yoga Studio,Flower Shop,Dessert Shop,Diner,Dive Bar,Donut Shop,Electronics Store
5,M.P.T.,72.892,4,Hotel,Resort,Business Service,Yoga Studio,Flower Shop,Dessert Shop,Diner,Dive Bar,Donut Shop,Electronics Store
31,Falkland Road,72.892,4,Hotel,Resort,Business Service,Yoga Studio,Flower Shop,Dessert Shop,Diner,Dive Bar,Donut Shop,Electronics Store
32,Mumbai Central,72.892,4,Hotel,Resort,Business Service,Yoga Studio,Flower Shop,Dessert Shop,Diner,Dive Bar,Donut Shop,Electronics Store
33,M A Marg,72.892,4,Hotel,Resort,Business Service,Yoga Studio,Flower Shop,Dessert Shop,Diner,Dive Bar,Donut Shop,Electronics Store
34,J.J.Hospital,72.892,4,Hotel,Resort,Business Service,Yoga Studio,Flower Shop,Dessert Shop,Diner,Dive Bar,Donut Shop,Electronics Store


In [144]:
Mumbai_merged.loc[Mumbai_merged['Cluster Labels'] == 5, Mumbai_merged.columns[[1] + list(range(5, Mumbai_merged.shape[1]))]]

,Neighborhood,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
20,Colaba,72.811,5,Convenience Store,Garden,Yoga Studio,Food,Dessert Shop,Diner,Dive Bar,Donut Shop,Electronics Store,Event Space
21,Holiday Camp,72.811,5,Convenience Store,Garden,Yoga Studio,Food,Dessert Shop,Diner,Dive Bar,Donut Shop,Electronics Store,Event Space
22,V.W.T.C.,72.811,5,Convenience Store,Garden,Yoga Studio,Food,Dessert Shop,Diner,Dive Bar,Donut Shop,Electronics Store,Event Space
23,Colaba Bazar,72.811,5,Convenience Store,Garden,Yoga Studio,Food,Dessert Shop,Diner,Dive Bar,Donut Shop,Electronics Store,Event Space
24,Asvini,72.811,5,Convenience Store,Garden,Yoga Studio,Food,Dessert Shop,Diner,Dive Bar,Donut Shop,Electronics Store,Event Space


In [145]:
Mumbai_merged.loc[Mumbai_merged['Cluster Labels'] == 6, Mumbai_merged.columns[[1] + list(range(5, Mumbai_merged.shape[1]))]]

,Neighborhood,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
121,Bandra West,73.875,6,Home Service,Dance Studio,Harbor / Marina,Flower Shop,Department Store,Dessert Shop,Diner,Dive Bar,Donut Shop,Electronics Store
133,Santacruz Central,73.875,6,Home Service,Dance Studio,Harbor / Marina,Flower Shop,Department Store,Dessert Shop,Diner,Dive Bar,Donut Shop,Electronics Store
134,Santacruz(West),73.875,6,Home Service,Dance Studio,Harbor / Marina,Flower Shop,Department Store,Dessert Shop,Diner,Dive Bar,Donut Shop,Electronics Store
135,Vakola,73.875,6,Home Service,Dance Studio,Harbor / Marina,Flower Shop,Department Store,Dessert Shop,Diner,Dive Bar,Donut Shop,Electronics Store
136,Santacruz(East),73.875,6,Home Service,Dance Studio,Harbor / Marina,Flower Shop,Department Store,Dessert Shop,Diner,Dive Bar,Donut Shop,Electronics Store
139,Vileparle Railway Station,73.875,6,Home Service,Dance Studio,Harbor / Marina,Flower Shop,Department Store,Dessert Shop,Diner,Dive Bar,Donut Shop,Electronics Store
140,Hanuman Road,73.875,6,Home Service,Dance Studio,Harbor / Marina,Flower Shop,Department Store,Dessert Shop,Diner,Dive Bar,Donut Shop,Electronics Store
141,Vileeparle (East),73.875,6,Home Service,Dance Studio,Harbor / Marina,Flower Shop,Department Store,Dessert Shop,Diner,Dive Bar,Donut Shop,Electronics Store
142,H.M.P. School,73.875,6,Home Service,Dance Studio,Harbor / Marina,Flower Shop,Department Store,Dessert Shop,Diner,Dive Bar,Donut Shop,Electronics Store
143,Andheri Railway Station,73.875,6,Home Service,Dance Studio,Harbor / Marina,Flower Shop,Department Store,Dessert Shop,Diner,Dive Bar,Donut Shop,Electronics Store


In [146]:
Mumbai_merged.loc[Mumbai_merged['Cluster Labels'] == 7, Mumbai_merged.columns[[1] + list(range(5, Mumbai_merged.shape[1]))]]

,Neighborhood,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
172,Kurla North,72.883,7,Jewelry Store,Yoga Studio,Food,Department Store,Dessert Shop,Diner,Dive Bar,Donut Shop,Electronics Store,Event Space
173,Kurla,72.883,7,Jewelry Store,Yoga Studio,Food,Department Store,Dessert Shop,Diner,Dive Bar,Donut Shop,Electronics Store,Event Space
174,Netajinagar,72.883,7,Jewelry Store,Yoga Studio,Food,Department Store,Dessert Shop,Diner,Dive Bar,Donut Shop,Electronics Store,Event Space


In [147]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Mumbai_merged['Latitude'], Mumbai_merged['Longitude'], Mumbai_merged['Neighborhood'], Mumbai_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters